## Utils


In [21]:
import random

In [23]:
LAMBDA = 1


class BikeParams:
    def __init__(self, quantum_security_level = LAMBDA, bike_variant = 1 | 2):
        if bike_variant in [1, 2]:
            if quantum_security_level == 1:
                self.n, self.r, self.w, self.t = 20326, 10163, 142, 134
            if quantum_security_level == 3:
                self.n, self.r, self.w, self.t = 39706, 19853, 206, 199
            if quantum_security_level == 5:
                self.n, self.r, self.w, self.t = 65498, 32749, 274, 264
        
        else:
            if quantum_security_level == 1:
                self.n, self.r, self.w, self.t = 22054, 11027, 134, 154
            if quantum_security_level == 3:
                self.n, self.r, self.w, self.t = 43366, 21683, 198, 226
            if quantum_security_level == 5:
                self.n, self.r, self.w, self.t = 72262, 36131, 266, 300
            
BIKE = BikeParams()


In [38]:


class Utils:

    def __init__(self) -> None:
        pass

    def odd_div_2(r = BIKE.r)->int:
        if (r // 2) % 2 == 0:
            return r//2 + 1
        return r//2




class BinaryPolynom:  # Polynom or Vector
    def __init__(self, r=BIKE.r) -> None:
        self.degree = r-1
        self.coefficients = [0 for i in range(self.degree+1)]
        pass

    def from_list(self, l):
        self.coefficients = [element for element in l]

    def get_random_polynom_with_weight(self, w = BIKE.w):
        self.coefficients[:w//2] = [1 for i in range(w//2)]
        random.shuffle(self.coefficients)

    def hamming_weight(self):
        return self.coefficients.count(1)

    def transpose(self):
        self.coefficients[1:] = self.coefficients[1:][::-1]
    
    def __str__(self) -> str:
        return "".join(str(bit) for bit in  self.coefficients)
    
    def __repr__(self) -> str:
        return str(self.coefficients)

    def __add__(a, b):
        c = BinaryPolynom(a.degree+1)
        for i in range(a.degree+1):
            c.coefficients[i] = a.coefficients[i] ^ b.coefficients[i]
        return c

    def __eq__(self, __value: object) -> bool:

        if not isinstance(__value, BinaryPolynom):
            return False
        if self.degree != __value.degree:
            return False
        
        for i in range(self.degree+1):
            if self.coefficients[i] != __value.coefficients[i]:
                return False
        return True
    
    def __sub__(a, b):
        c = BinaryPolynom(a.degree+1)
        for i in range(a.degree+1):
            c.coefficients[i] = a.coefficients[i] ^ b.coefficients[i]
        return c

    def __mul__(a, b):
        c = BinaryPolynom(a.degree+1)

        for k in range(b.degree+1):
            for j in range(k+1):
                c.coefficients[k] ^= a.coefficients[j]&b.coefficients[k-j]
            
        return c
        


In [52]:
a = BinaryPolynom(25)
a.get_random_polynom_with_weight(17)

b = BinaryPolynom(25)
b.get_random_polynom_with_weight(16)



print(a)
a.transpose()
print(a)


0000101001011000010001010
0010100010000110100101000


## Key Generation

In [26]:
h_0 = BinaryPolynom()
h_0.get_random_polynom_with_weight()

h_1 = BinaryPolynom()
h_1.get_random_polynom_with_weight()

secret_key = (h_0, h_1)

g = BinaryPolynom()
g.get_random_polynom_with_weight(Utils.odd_div_2()) # BIKE 3

public_key = (h_1*g, h_0*g) # BIKE 1 & BIKE 3

## Encryption

In [27]:
import hashlib

# Generer e0 et e1 deux polynomes tels que |e0| + |e1| = t

def brute_force_generator(t = BIKE.t):

    is_ok = False
    e0, e1 = BinaryPolynom(), BinaryPolynom()
    while not is_ok:
        e0.get_random_polynom_with_weight(random.randint(1, t))
        e1.get_random_polynom_with_weight(random.randint(1, t))
        is_ok = (e0.hamming_weight() + e1.hamming_weight() == t)
    
    return e0, e1

def brute_force_generator_2(t = BIKE.t):
    k = random.randint(1, t)
    e0, e1 = BinaryPolynom(), BinaryPolynom()
    e0.get_random_polynom_with_weight(k)
    e1.get_random_polynom_with_weight(t-k)
    
    return e0, e1

def K(e_0, e_1):
    bit_string = str(e_0) + str(e_1)
    hash = hashlib.sha3_256()
    hash.update(bit_string.encode())

    return hash.digest()[:32]

In [28]:
m = BinaryPolynom()
e_0, e_1 = brute_force_generator_2()

c_0, c_1 = m*public_key[0] + e_0, m*public_key[1] + e_1  # BIKE 1

symmetric_key = K(e_0, e_1)

b'\t\x85\xe5\ra\xe5\x1c>\xce\xfa\x93\xd7\x01\xa3R\xbe\x9e\x17\x96\x8f~\xf0\x9b\xaf\xfe5h\x16\xc8\xa0\xff\xcc'

## Decription

In [ ]:
def decode(s, h_0, h_1, u):
    

    

In [ ]:
# calcule du syndrome
s = c_0*h_0 + c_1*h_1
u = 0

# Decoder s pour retrouver (e_0_prime, e_1_prime)



symmetric_key_decapsuled = bytes()
e_0_prime, e_1_prime = BinaryPolynom(), BinaryPolynom()
if e_0_prime.hamming_weight()  + e_1_prime.hamming_weight() != BIKE.t:
    print("Decapsulation error !")
    
else:
    symmetric_key_decapsuled = K(e_0_prime, e_1_prime)
 

## BIKE Entry point

In [ ]:

# Fonction principale pour exécuter les fonctionnalités de BIKE
def main():
    # Générer les clés publiques et privées
    public_key, private_key = key_generation.generate_key_pair()

    # Message à chiffrer
    message = "Hello, World!"

    # Chiffrement du message avec la clé publique
    ciphertext = encryption.encrypt(message, public_key)

    # Déchiffrement du message chiffré avec la clé privée
    decrypted_message = decryption.decrypt(ciphertext, private_key)

    # Affichage des résultats
    print("Message clair :", message)
    print("Message chiffré :", ciphertext)
    print("Message déchiffré :", decrypted_message)

# Exécution de la fonction principale
if __name__ == "__main__":
    main()